In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import gc
gc.collect()

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
cnt = 0
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        cnt+=1
print(cnt)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
train_df=pd.read_csv('../input/feedback-prize-effectiveness/train.csv')
train_df.info()

In [ ]:
train_df.head()

In [ ]:
print(train_df['discourse_effectiveness'].value_counts())
print('\n\n')
print(train_df['discourse_type'].value_counts())

### data pre processing

In [ ]:
train_df['all_text'] = train_df['discourse_text']+' '+train_df['discourse_type']
train_df.head()

In [ ]:
temp = train_df['all_text'].values

In [ ]:
temp[:10]

##### To do -
- lower case
- punctuation
- stop words
- remove numbers
- lemma ?

In [ ]:
import re
import string
from nltk.corpus import stopwords

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\W+\s*',' ',text)
    text = text.replace('_',' ')
    text = re.sub(r'\d+',' ',text)
    text = re.sub(r'\s\s+',' ',text)
    stops =  list(set(stopwords.words('english'))) 
    stops.extend(list(string.punctuation))
    text = [word.strip() for word in text.split() if word.strip() not in stops]
    
    return ' '.join(text)
    


In [ ]:
print(temp[6])
print('=============')
clean_text(temp[6])

In [ ]:
train_df['clean_text'] = train_df['all_text'].apply(lambda x: clean_text(x))
train_df.head()

In [ ]:
train_df.drop(['discourse_id','essay_id','discourse_text','discourse_type','all_text'],inplace=True,axis=1)

In [ ]:
train_df.head()

- prep target values

In [ ]:
tar_map = {"Ineffective":0, "Adequate":1,"Effective":2}
train_df["target"] = train_df["discourse_effectiveness"].map(tar_map)
train_df.head()

In [ ]:
print(train_df['target'].value_counts())
print(tar_map)
print(train_df['discourse_effectiveness'].value_counts())

In [ ]:
train_df.drop(['discourse_effectiveness'],inplace=True,axis=1)
train_df.head()

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train_,y_test_ = train_test_split(train_df['clean_text'],train_df['target'],random_state=1234,test_size=0.25)

print(x_train.shape,y_train_.shape)
print(x_test.shape,y_test_.shape)

In [ ]:
y_train = pd.get_dummies(y_train_).values
y_test = pd.get_dummies(y_test_).values

print(x_train.shape,y_train_.shape)
print(x_test.shape,y_test_.shape)

In [ ]:
# x_train.values

### Model prep

In [ ]:
import torch
import torch.nn as nn
import transformers
from transformers import EarlyStoppingCallback
from transformers import Trainer,TrainingArguments

from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.utils.clip_grad import clip_grad_norm


torch.cuda.is_available()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

In [ ]:
model_path = '../input/huggingface-bert/bert-base-uncased'
max_len = 128

In [ ]:
# class my_model(nn.Module):
#     def __init__(self,model_path):
#         super(my_model,self).__init__()
#         self.model_path = model_path
#         self.model = transformers.AutoModel.from_pretrained(self.model_path)
#         self.fc_layer = nn.Sequential(
#             nn.Dropout(0.3),
#             nn.Linear(self.model.get_word_embedding_dimension(),256,bias=True),
#             nn.LeakyReLU(),
#             nn.Linear(256,3),
#             nn.Sigmoid()
#         )
#     def forward(self,ids,mask,token_type_ids):
#         out = self.bert(input_ids=ids,attention_mask=mask,token_type_ids=token_type_ids,return_dict=True)
#         pooler_output = out.get('pooler_output')
#         bo = self.fc_layer(pooler_output)
#         return bo
     

In [ ]:
# class my_dataset_train:
#     def __init__(self,text,label,tokenizer,max_len):
#         self.text=text
#         self.label=label
#         self.tokenizer = tokenizer
#         self.max_len = max_len
        
#     def __len__(self):
#         return len(self.text)
    
#     def __getitem__(self,idx):
#         text_ = str(self.text[idx])
#         label_ = self.label[idx]
        
#         inputs = self.tokenizer(
#             text_,
#             add_special_tokens=True,
#             padding='max_length',
#             truncation=True,
#             max_length=self.max_len,
#             return_attention_mask=True
#         )
        
#         ids = inputs['input_ids']
#         token_type_ids = inputs["token_type_ids"]
#         mask = inputs['attention_mask']
        
#         return {
#             "ids": torch.tensor(ids,dtype=torch.long),
#             "mask": torch.tensor(mask,dtype=torch.long),
#             "token_type_ids": torch.tensor(token_type_ids,dtype=torch.long),
#             "targets": torch.tensor(label_,dtype=torch.float),
#         }

In [ ]:
# tokenizer = transformers.AutoTokenizer.from_pretrained(bert_path)

# # Training dataset prep

# train_text = list(x_train.values)

# train_dataset = my_dataset_train(
#     text = train_text,
#     label = y_train,
#     tokenizer=tokenizer ,
#     max_len=max_len
# )

# train_data_loader = torch.utils.data.DataLoader(train_dataset,batch_size=train_batch_size,shuffle=True)

# # validation dataset prep
# val_text1 = list(x_test['target'].values)
# val_text2 = list(x_test['sen1'].values)
# val_label = list(y_test.values)

# valid_dataset = my_dataset_train(
#     text1 = val_text1,
#     text2 = val_text2,
#     label = val_label,
#     tokenizer=tokenizer,
#     max_len=max_len
# )

# valid_data_loader = torch.utils.data.DataLoader(valid_dataset,batch_size=train_batch_size,shuffle=True)

    

In [ ]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         print(self.labels,type(self.labels))
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
tokenizer = BertTokenizer.from_pretrained(model_path)

X_train_tokenized = tokenizer(list(x_train.values),add_special_tokens=True,padding='max_length',truncation=True,max_length=max_len,return_attention_mask=True)
X_val_tokenized = tokenizer(list(x_test.values),add_special_tokens=True,padding='max_length',truncation=True,max_length=max_len,return_attention_mask=True)

train_dataset = Dataset(X_train_tokenized, list(y_train_))
val_dataset = Dataset(X_val_tokenized, list(y_test_))


In [ ]:
from sklearn.metrics import classification_report
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    
    result = classification_report(labels,pred,output_dict=True)

    return result


In [ ]:
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=3)

In [ ]:
# Define Trainer
args = TrainingArguments(
    output_dir="./output/",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    num_train_epochs=6,
    save_strategy="epoch",
    seed=0,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

In [ ]:
trainer.train()

### Inference

In [ ]:
test_data = pd.read_csv('../input/feedback-prize-effectiveness/test.csv')
test_data.head()

In [ ]:
test_data['all_text'] = test_data['discourse_text']+' '+test_data['discourse_type']
test_data['clean_text'] = test_data['all_text'].apply(lambda x: clean_text(x))
test_data.drop(['essay_id','discourse_text','discourse_type','all_text'],inplace=True,axis=1)
test_data.head()

In [ ]:
test_x = list(test_data["clean_text"])
test_x_tokenized = tokenizer(list(test_data["clean_text"]),add_special_tokens=True,padding='max_length',truncation=True,max_length=max_len,return_attention_mask=True)

test_dataset_ = Dataset(test_x_tokenized)


In [ ]:
# # Load trained model
# model_path_trained = "output/checkpoint-50000"
# model = BertForSequenceClassification.from_pretrained(model_path_trained, num_labels=3)

# test_trainer = Trainer(model)

# Make prediction
raw_pred, _, _ = trainer.predict(test_dataset_)


In [ ]:
raw_pred.shape, test_data.shape

In [ ]:
sub = pd.read_csv('../input/feedback-prize-effectiveness/sample_submission.csv')
sub.head()

In [ ]:
sub.shape

In [ ]:
raw_pred

In [ ]:
x=torch.sigmoid(torch.tensor(raw_pred))
x

In [ ]:
final = []
ids = list(test_data['discourse_id'].values)
for i in range(len(x)):
    t = [ids[i]] + x[i].tolist()
    final.append(t)
df_ = pd.DataFrame(final,columns=list(sub.columns))
df_.head()

In [ ]:
df_.to_csv('submission.csv',index=False)